# Deconvolution Header File Generator

This notebook generates `deconv_top.hpp` files with configurable parameters for HLS deconvolution implementations.

## Parameters:
- **S**: Stride
- **K**: Kernel Size
- **H**: Input Feature Map Height
- **W**: Input Feature Map Width
- **CI**: Input Channels
- **CO**: Output Channels

## 1. Import Required Libraries

In [26]:
import numpy as np
import itertools
from pathlib import Path
import os
from typing import List, Tuple, Dict

## 2. Define Parameter Configuration Class

In [27]:
class DeconvConfig:
    """Configuration class for deconvolution parameters"""
        

    def __init__(self, K: int, S: int, H: int, W: int, CI: int, CO: int, P: int = None):
        self.K = K    # Kernel size
        self.S = S    # Stride
        self.H = H    # Input height
        self.W = W    # Input width
        self.CI = CI  # Input channels
        self.CO = CO  # Output channels
        self.P = P if P is not None else K - S  # Padding (derived if not provided)
        
    def validate(self) -> bool:
        """Validate parameter constraints"""
        if self.K <= 0 or self.S <= 0 or self.H <= 0 or self.W <= 0:
            return False
        if self.CI <= 0 or self.CO <= 0:
            return False
        if self.P < 0:
            return False
        return True
    
    def __str__(self):
        return f"K={self.K}, S={self.S}, H={self.H}, W={self.W}, CI={self.CI}, CO={self.CO}, P={self.P}"

## 3. Generate Kernel Data Arrays

In [28]:
def generate_kernel_weights(config: DeconvConfig, pe: int = 1, simd: int = 1) -> str:
    """Generate kernel weight array in C++ format"""
    
    # Calculate array dimensions
    outer_dim = (config.CO // pe) * config.K * config.K * (config.CI // simd)
    
    kernel_lines = []
    kernel_lines.append(f"static TW const  KERNEL[{outer_dim}][{pe}][{simd}] = {{")
    
    # Generate weight values
    weight_counter = 0
    for co_group in range(config.CO // pe):
        for k_row in range(config.K):
            for k_col in range(config.K):
                for ci_group in range(config.CI // simd):
                    # Create the inner arrays
                    pe_values = []
                    for p in range(pe):
                        simd_values = []
                        for s in range(simd):
                            # Generate hex values (can be customized)
                            hex_val = f"0x{weight_counter:02x}"
                            simd_values.append(hex_val)
                            weight_counter += 1
                        if simd == 1:
                            pe_values.append(f"{{{simd_values[0]},}}")
                        else:
                            pe_values.append(f"{{{','.join(simd_values)},}}")
                    
                    if pe == 1:
                        line = f"\t{{{pe_values[0]}}},"
                    else:
                        line = f"\t{{{','.join(pe_values)}}},"
                    kernel_lines.append(line)
    
    kernel_lines.append("};")
    # print("\n".join(kernel_lines))
    return "\n".join(kernel_lines)

def generate_pe_simd_configs(config: DeconvConfig) -> List[Tuple[int, int]]:
    """Generate valid PE and SIMD configurations"""
    configs = []
    
    # Find divisors of CO for PE
    pe_options = [i for i in range(1, config.CO + 1) if config.CO % i == 0]
    
    # Find divisors of CI for SIMD  
    simd_options = [i for i in range(1, config.CI + 1) if config.CI % i == 0]
    
    # Common configurations
    for pe in pe_options[:3]:  # Limit to first 3 options
        for simd in simd_options[:2]:  # Limit to first 2 options
            configs.append((pe, simd))
    
    return configs if configs else [(1, 1)]

## 4. Create Header File Template

In [29]:
def load_weights_from_csv(weights_path) -> List[int]:
    """Load flat list of weights from a CSV file. Supports comma/whitespace separated,
    decimal or hex (0x..) values. Non-numeric tokens are ignored."""
    text = Path(weights_path).read_text().strip()
    tokens = []
    for line in text.splitlines():
        parts = [p for p in line.replace(',', ' ').split() if p]
        tokens.extend(parts)

    values = []
    for t in tokens:
        try:
            v = int(t, 16) if t.lower().startswith("0x") else int(float(t))
            values.append(v & 0xFF)  # clamp to 8-bit
        except Exception:
            # ignore non-numeric tokens (headers, etc.)
            continue
    return values

def generate_kernel_weights(config: DeconvConfig, pe: int = 1, simd: int = 1, weights: List[int] = None) -> str:
    """Generate kernel weight array in C++ format. If weights is provided, use it;
    otherwise generate a simple incremental pattern."""
    outer_dim = (config.CO // pe) * config.K * config.K * (config.CI // simd)
    total_elems = outer_dim * pe * simd

    if weights is None or len(weights) == 0:
        # fallback: incremental pattern
        weights = list(range(total_elems))
    else:
        # adjust length to match required elements
        if len(weights) < total_elems:
            weights = weights + [0] * (total_elems - len(weights))
        elif len(weights) > total_elems:
            weights = weights[:total_elems]

    kernel_lines = []
    kernel_lines.append(f"static TW const  KERNEL[{outer_dim}][{pe}][{simd}] = {{")

    idx = 0
    for _co_group in range(config.CO // pe):
        for _k_row in range(config.K):
            for _k_col in range(config.K):
                for _ci_group in range(config.CI // simd):
                    pe_values = []
                    for _p in range(pe):
                        simd_values = []
                        for _s in range(simd):
                            hex_val = f"0x{(weights[idx] & 0xFF):02x}"
                            simd_values.append(hex_val)
                            idx += 1
                        if simd == 1:
                            pe_values.append(f"{{{simd_values[0]},}}")
                        else:
                            pe_values.append(f"{{{','.join(simd_values)},}}")

                    if pe == 1:
                        line = f"\t{{{pe_values[0]}}},"
                    else:
                        line = f"\t{{{','.join(pe_values)}}},"
                    kernel_lines.append(line)

    kernel_lines.append("};")
    return "\n".join(kernel_lines)

def generate_header_file(config: DeconvConfig, pe_simd_configs: List[Tuple[int, int]], weights_path: str = None) -> str:
    """Generate complete header file content. If weights_path is provided, load weights
    from CSV and pass them to generate_kernel_weights."""
    header_template = f"""#ifndef DECONV_TOP_HPP
#define DECONV_TOP_HPP

#include <ap_int.h>
#include <hls_stream.h>
#include <hls_vector.h>

constexpr unsigned  K = {config.K};		// kernel Size
constexpr unsigned  S = {config.S}; 		// stride
constexpr unsigned  P = {config.P};		// padding
constexpr unsigned  H = {config.H};		// IFM height
constexpr unsigned  W = {config.W};		// IFM Width
constexpr unsigned  CI = {config.CI};		// input channels
constexpr unsigned  CO = {config.CO};		// output channels

using  TW = ap_uint< 8>;
using  TI = ap_uint< 4>;
using  TO = ap_uint<16>;

"""

    provided_weights = load_weights_from_csv(weights_path) if weights_path else None

    config_sections = []
    for i, (pe, simd) in enumerate(pe_simd_configs):
        if i == 0:
            config_sections.append("#if 1\n")
        else:
            config_sections.append("#else\n")

        config_sections.append(f"constexpr unsigned  PE   = {pe};\n")
        config_sections.append(f"constexpr unsigned  SIMD = {simd};\n")
        config_sections.append("")

        kernel_weights = generate_kernel_weights(config, pe, simd, weights=provided_weights)
        config_sections.append(kernel_weights)
        config_sections.append("")

    if len(pe_simd_configs) > 1:
        config_sections.append("#endif")

    function_decl = """
void deconv_top(
    hls::stream<hls::vector<TI, SIMD>> &src,
    hls::stream<hls::vector<TO, PE>>   &dst
);

#endif"""

    full_content = header_template + "\n".join(config_sections) + function_decl
    return full_content

## 5. Generate Multiple Configurations

In [30]:
import csv

# Define various parameter combinations to generate
# configurations = [
#     # Original configuration
#     DeconvConfig(K=4, S=2, H=6, W=6, CI=1, CO=2),
    
#     # Small kernel configurations
#     DeconvConfig(K=3, S=1, H=8, W=8, CI=2, CO=4),
#     DeconvConfig(K=2, S=1, H=4, W=4, CI=1, CO=1),
    
#     # Large kernel configurations
#     DeconvConfig(K=5, S=2, H=10, W=10, CI=4, CO=8),
#     DeconvConfig(K=7, S=3, H=14, W=14, CI=3, CO=6),
    
#     # Different channel configurations
#     DeconvConfig(K=4, S=2, H=6, W=6, CI=2, CO=4),
#     DeconvConfig(K=4, S=2, H=6, W=6, CI=4, CO=8),
    
#     # Non-square input sizes
#     DeconvConfig(K=3, S=1, H=6, W=12, CI=2, CO=2),
#     DeconvConfig(K=4, S=2, H=8, W=4, CI=1, CO=2),
# ]
configurations=[]

# Load configurations from CSV file

deconv_data_dir = Path("/mnt/Crucial/WorkspaceB/finn_hls/DeConv_benchmark/deconv_data")
config_csv_path = Path(f"{deconv_data_dir}/configs/deconv_configs.csv")
exp_data_dir = Path(f"{deconv_data_dir}/exp_data")

if config_csv_path.exists():

    print(f"Loading configurations from {config_csv_path}...")
    with open(config_csv_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                config = DeconvConfig(
                    K=int(row['kernel_size']),
                    S=int(row['stride']),
                    H=int(row['input_size']),
                    W=int(row['input_size']),
                    CI=int(row['in_channels']),
                    CO=int(row['out_channels']),
                    P=int(row['padding'])
                )
                configurations.append(config)
                print(f"  Loaded: {config}")
            except (KeyError, ValueError) as e:
                print(f"  Skipping invalid row: {row} - Error: {e}")
    
    print(f"Total configurations (including CSV): {len(configurations)}\n")
else:
    print(f"No CSV file found at {csv_path}, using predefined configurations only.\n")


# Validate and filter configurations
valid_configs = []
for config in configurations:
    print(f"Validating configuration: {config}...")
    if config.validate():
        valid_configs.append(config)
        print(f"✓ Valid: {config}")
    else:
        print(f"✗ Invalid: {config}")

print(f"\\nTotal valid configurations: {len(valid_configs)}")

weights_files = []
config_groups = []

# File suffixes to look for
suffixes = {
    'weights': '_weights.csv',
    'input': '_input.csv',
    'output': '_output.csv',
}

for cfg in valid_configs:
    base = f"deconv_{cfg.H}x{cfg.W}_in{cfg.CI}_out{cfg.CO}_k{cfg.K}_s{cfg.S}_p{cfg.P}"
    files = {}
    all_present = True

    for kind, suff in suffixes.items():
        fpath = exp_data_dir / f"{base}{suff}"
        if fpath.exists():
            files[kind] = str(fpath)
            if kind == 'weights':
                weights_files.append((cfg, str(fpath)))
            print(f"Found {kind} file for config: {cfg} -> {fpath}")
        else:
            files[kind] = None
            all_present = False
            print(f"No {kind} file for config: {cfg}")

    config_groups.append({
        'config': cfg,
        'files': files,
        'all_present': all_present
    })

print(f"\nTotal configs with weights files: {len(weights_files)}")
print(f"Total config groups: {len(config_groups)}")
print(f"Config groups with all files present: {sum(1 for g in config_groups if g['all_present'])}")

print(config_groups)

Loading configurations from /mnt/Crucial/WorkspaceB/finn_hls/DeConv_benchmark/deconv_data/configs/deconv_configs.csv...
  Loaded: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2
  Loaded: K=3, S=1, H=3, W=3, CI=1, CO=3, P=1
Total configurations (including CSV): 2

Validating configuration: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2...
✓ Valid: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2
Validating configuration: K=3, S=1, H=3, W=3, CI=1, CO=3, P=1...
✓ Valid: K=3, S=1, H=3, W=3, CI=1, CO=3, P=1
\nTotal valid configurations: 2
Found weights file for config: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2 -> /mnt/Crucial/WorkspaceB/finn_hls/DeConv_benchmark/deconv_data/exp_data/deconv_3x3_in1_out3_k3_s1_p2_weights.csv
Found input file for config: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2 -> /mnt/Crucial/WorkspaceB/finn_hls/DeConv_benchmark/deconv_data/exp_data/deconv_3x3_in1_out3_k3_s1_p2_input.csv
Found output file for config: K=3, S=1, H=3, W=3, CI=1, CO=3, P=2 -> /mnt/Crucial/WorkspaceB/finn_hls/DeConv_benchmark/deconv_data/exp_

## 6. Export Generated Files

In [31]:
# Create output directory
output_dir = Path("generated_configs")
output_dir.mkdir(exist_ok=True)

generated_files = []

# Generate header files for each configuration
for i, de_config in enumerate(config_groups):
    config = de_config['config']
    weights_file = de_config['files']['weights']
    # Generate PE/SIMD configurations
    pe_simd_configs = generate_pe_simd_configs(config)
    
    # Generate header content and ensure padding is part of the filename
    header_content = generate_header_file(config, pe_simd_configs, weights_path=weights_file)
    filename = f"deconv_top_K{config.K}_S{config.S}_H{config.H}_W{config.W}_CI{config.CI}_CO{config.CO}_P{config.P}.hpp"
    filepath = output_dir / filename
    header_content = generate_header_file(config, pe_simd_configs, weights_path=weights_file)
    

    # Write to file
    with open(filepath, 'w') as f:
        f.write(header_content)
    
    generated_files.append({
        'filename': filename,
        'config': str(config),
        'pe_simd_configs': pe_simd_configs,
        'filepath': str(filepath)
    })
    
    print(f"Generated: {filename}")


    # Create a selector header that includes one of the generated config headers
    selector_path = output_dir / "deconv_top.hpp"

    lines = []
    lines.append("// Auto-generated selector header for deconvolution configurations")
    lines.append("// Usage: define one of the following macros before including this file:")
    for idx, info in enumerate(generated_files):
        param_tag = info['filename'].replace("deconv_top_", "").replace(".hpp", "")
        lines.append(f"//   - DECONV_CFG_IDX_{idx}")
        lines.append(f"//   - DECONV_CFG_{param_tag}")
    lines.append("")

    guard = "DECONV_TOP_SELECTOR_HPP"
    lines.append(f"#ifndef {guard}")
    lines.append(f"#define {guard}")
    lines.append("")

    if generated_files:
        for idx, info in enumerate(generated_files):
            param_tag = info['filename'].replace("deconv_top_", "").replace(".hpp", "")
            macro_idx = f"DECONV_CFG_IDX_{idx}"
            macro_param = f"DECONV_CFG_{param_tag}"
            cond = f"defined({macro_idx}) || defined({macro_param})"
            prefix = "#if" if idx == 0 else "#elif"
            lines.append(f"{prefix} {cond}")
            lines.append(f'#include "{info["filename"]}"')
        # Default to the first config if none specified
        lines.append("#else")
        lines.append(f'#include "{generated_files[0]["filename"]}"')
        lines.append("#endif")
    else:
        lines.append('#error "No configuration headers were generated. Please generate configurations first."')

    lines.append("")
    lines.append(f"#endif // {guard}")

    with open(selector_path, "w") as f:
        f.write("\n".join(lines))

    print(f"Selector header generated: {selector_path}")

print(f"\\n📁 All files saved to: {output_dir.absolute()}")
print(f"📊 Total files generated: {len(generated_files)}")

Generated: deconv_top_K3_S1_H3_W3_CI1_CO3_P2.hpp
Selector header generated: generated_configs/deconv_top.hpp
Generated: deconv_top_K3_S1_H3_W3_CI1_CO3_P1.hpp
Selector header generated: generated_configs/deconv_top.hpp
\n📁 All files saved to: /mnt/Crucial/WorkspaceB/finn_hls/DeConv_hls_benchmark/generated_configs
📊 Total files generated: 2


In [7]:
# Display summary table
print("\\n" + "="*80)
print("GENERATED CONFIGURATIONS SUMMARY")
print("="*80)
print(f"{'Filename':<45} {'Parameters':<35} {'PE/SIMD'}")
print("-"*80)

for file_info in generated_files:
    pe_simd_str = ", ".join([f"({pe},{simd})" for pe, simd in file_info['pe_simd_configs']])
    print(f"{file_info['filename']:<45} {file_info['config']:<35} {pe_simd_str}")

print("="*80)
print("\\n🔧 Next Steps:")
print("  1. Run HLS project generation: ./manage_hls_projects.sh generate")
print("  2. Run C simulation: ./manage_hls_projects.sh csim")
print("  3. Run synthesis: ./manage_hls_projects.sh synthesize")
print("  4. Run co-simulation: ./manage_hls_projects.sh cosim")
print("  5. Gather output files: ./manage_hls_projects.sh gather-outputs")
print("  6. Copy golden reference files: ./manage_hls_projects.sh gather-golden")
print("  7. Compare results with golden references: ./manage_hls_projects.sh compare-results")
print("\\n📁 Scripts are organized in the 'scripts/' folder")
print("📁 Generated configs are in the 'generated_configs/' folder")
print("📁 All configurations now include padding parameter (P) in filename")

\n================================================================================
GENERATED CONFIGURATIONS SUMMARY
Filename                                      Parameters                          PE/SIMD
--------------------------------------------------------------------------------
deconv_top_K3_S1_H5_W5_CI1_CO3_P2.hpp         K=3, S=1, H=5, W=5, CI=1, CO=3, P=2 (1,1), (3,1)
deconv_top_K3_S1_H5_W5_CI1_CO3_P1.hpp         K=3, S=1, H=5, W=5, CI=1, CO=3, P=1 (1,1), (3,1)
\n🔧 Next Steps:
  1. Run HLS project generation: ./manage_hls_projects.sh generate
  2. Run C simulation: ./manage_hls_projects.sh csim
  3. Run synthesis: ./manage_hls_projects.sh synthesize
  4. Run co-simulation: ./manage_hls_projects.sh cosim
  5. Gather output files: ./manage_hls_projects.sh gather-outputs
  6. Copy golden reference files: ./manage_hls_projects.sh gather-golden
  7. Compare results with golden references: ./manage_hls_projects.sh compare-results
\n📁 Scripts are organized in the 'scripts/' folder


## 🔧 Next Steps: Complete HLS Workflow with Golden Results Validation

After generating the configuration files, follow this workflow:

### 1. **Project Organization** 
All automation scripts are now organized in the `scripts/` folder:
- `scripts/validate_hls_setup.tcl` - Environment validation
- `scripts/generate_hls_projects.tcl` - Project generation  
- `scripts/demo_hls_projects.tcl` - Demonstration workflow
- `scripts/test_hls_basic.tcl` - Basic testing

### 2. **HLS Development Workflow**
```bash
# 1. Validate HLS environment
./manage_hls_projects.sh validate

# 2. Generate HLS projects for all configurations
./manage_hls_projects.sh generate

# 3. Run complete workflow (C simulation → Synthesis → Co-simulation)
./manage_hls_projects.sh demo

# 4. Check project status
./manage_hls_projects.sh status
```

### 3. **Results Collection and Validation**
```bash
# 5. Gather HLS simulation output files with PE/SIMD naming
./manage_hls_projects.sh gather-outputs

# 6. Copy golden reference files from experimental data
./manage_hls_projects.sh gather-golden

# 7. Compare HLS outputs against golden references
./manage_hls_projects.sh compare-results
```

### 4. **Results Analysis**
The comparison generates:
- **Detailed Report**: `comparison_results/comparison_report.txt` - Line-by-line comparison details
- **CSV Summary**: `comparison_results/comparison_summary.csv` - Structured comparison results
- **Success Metrics**: Overall pass/fail rates and configuration-specific results

### 5. **Directory Structure**
```
📁 Project Structure:
├── manage_hls_projects.sh          # Main automation script
├── deconv_generator.ipynb          # This configuration generator
├── scripts/                        # HLS automation scripts
├── generated_configs/              # Generated .hpp files
├── outputs/                        # HLS simulation results (PE/SIMD named)
├── golden_results/                 # Golden reference files
├── comparison_results/             # Validation reports
└── hls_projects/                   # Generated HLS projects
```

### 6. **Additional Commands**
```bash
# Individual operations
./manage_hls_projects.sh csim        # C simulation only
./manage_hls_projects.sh synth       # Synthesis only  
./manage_hls_projects.sh cosim       # Co-simulation only
./manage_hls_projects.sh clean       # Clean all projects
./manage_hls_projects.sh list        # List configurations
./manage_hls_projects.sh help        # Show help
```

**🎯 The workflow ensures synthesis completes before co-simulation and provides comprehensive validation against golden reference data.**